In [ ]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
import numpy as np
import os
import cv2
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import Xception, InceptionV3, ResNet50, EfficientNetB0
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# # Google colab connection
# from google.colab import drive
# drive.mount('/content/drive')
# !unzip "/content/drive/MyDrive/Dataset/data.zip"

In [ ]:
# Enable GPU acceleration
gpus = tf.config.list_logical_devices('GPU')
stg = tf.distribute.MirroredStrategy(gpus)
location = '/content/data'
no_of_classes = 38
batch_size = 32
size = 224

In [ ]:
# Data generators
picture_size = (size, size)
train_datagen = ImageDataGenerator(validation_split=0.2)
train_set = train_datagen.flow_from_directory(
    directory=location,
    target_size=picture_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed=22
)

validation_set = train_datagen.flow_from_directory(
    directory=location,
    target_size=picture_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    seed=22
)


In [ ]:
# Function to build models
def build_model(base_model):
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs)
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(38, activation='softmax')(x)  # Assuming 38 classes
    
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(lr=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Load pre-trained models
models = [
    Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
]

# Build and compile models  
ensemble_models = [build_model(model) for model in models]

In [ ]:
# Callbacks
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
callbacks = [lr_scheduler, early_stopping]

In [ ]:
# Training models
histories = []
for i, model in enumerate(ensemble_models):
    print(f"Training Model {i+1}...")
    history = model.fit(train_set,
                        epochs=10,  # Adjust as needed
                        validation_data=validation_set,
                        callbacks=callbacks,
                        steps_per_epoch=len(train_set),
                        validation_steps=len(validation_set))
    histories.append(history)

    # Plotting loss per epoch
    plt.figure(figsize=(8, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Model {i+1} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Plotting accuracy per epoch
    plt.figure(figsize=(8, 6))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Model {i+1} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()   

In [ ]:
# Evaluate ensemble models
test_data_dir = location
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=picture_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Generate predictions for each model and collect them in a list
ensemble_predictions = []
for model in ensemble_models:
    predictions = model.predict(test_generator)
    ensemble_predictions.append(predictions)

# Compute the average predictions of the ensemble
ensemble_predictions = np.mean(ensemble_predictions, axis=0)

# Calculate metrics
y_true = test_generator.classes
y_pred = np.argmax(ensemble_predictions, axis=1)

# Classification Report
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()  

In [ ]:
# ROC AUC Score (for multi-class classification, average='macro' is used)
roc_auc = roc_auc_score(tf.keras.utils.to_categorical(y_true), ensemble_predictions, average='macro')
print("Overall ROC AUC Score:", roc_auc)

# ROC Curve
fpr, tpr, _ = roc_curve(tf.keras.utils.to_categorical(y_true).ravel(), ensemble_predictions.ravel())
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.5f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve')
plt.legend(loc="lower right")
plt.show()